In [1]:
import matplotlib.pyplot as plt
import nuclide_chart_plot as ncp
import numpy as np
import ipywidgets as widgets
from pyne.material import Material
from pyne import nucname
all_nuclide_list = ncp.get_nuclide_list()
u_enrich = Material({92235:0.05, 92238:0.95})
%matplotlib inline

In [2]:
mox_data = {}
fuel_types = 6 # simpler than detecting empty columns in table

# manual parsing of CSV to avoid learning how to get the headers & conversions right
with open('mox.csv') as mox_data_file:
    mox_data_file.readline()
    fuels = mox_data_file.readline().strip().split(',')[1:fuel_types*2+1]
    for fuel in fuels:
        mox_data[fuel] = {}
    for line in mox_data_file.readlines():
        tokens = line.split(',')
        try:
            nuclide = nucname.id(tokens[0])
            for fuel, comp in zip(fuels, [float(x) for x in tokens[1:fuel_types*2+1]]):
                mox_data[fuel][nuclide] = comp
        except:
            # skip rows that don't represent specific nuclides
            #print(f"skipping {tokens[0]}")
            pass

mox_materials = { fuel: Material(comp) for fuel, comp in mox_data.items()}

In [3]:
def interact_plot(material, decay_time, response):
    seconds_per_year = 365.25*24*60*60
    ncp.nuclide_chart_plot(material, all_nuclide_list, decay_time=decay_time*seconds_per_year, response=response)
    
style = {'description_width': 'initial'}
decay_time_slider = widgets.FloatLogSlider(value=-3, base=10, min=-3, max=8, step=0.2, description="Decay Time [years]", style=style)
response_choice = widgets.Dropdown(options=["inventory", "decay_heat", "dose"], description="Response to Plot", value="inventory", style=style)
material_choice = widgets.Dropdown(options=mox_materials, description="Material to plot", style=style)

In [4]:
widgets.interact(interact_plot, material=material_choice, decay_time=decay_time_slider, response=response_choice)

interactive(children=(Dropdown(description='Material to plot', options={'MOX-PN fresh': pyne.material.Material…

<function __main__.interact_plot(material, decay_time, response)>